In [13]:
import requests 
import json
from os.path import expanduser 
from requests.auth import HTTPBasicAuth

def get_auth () -> requests.Session:
    path = expanduser('~/brain_credentials_copy.txt')
    with open(path) as f:
        credentials = json.load(f)
    username,password = credentials['username'], credentials['password']

    sess = requests.Session ()

    sess.auth = HTTPBasicAuth (username, password)

    response = sess.post('https://api.worldquantbrain.com/authentication')

    print (response.status_code)
    print (response. json ())
    return sess

sess = get_auth ()

201
{'user': {'id': 'FP27330'}, 'token': {'expiry': 14400.0}, 'permissions': ['TUTORIAL']}


In [15]:
import pandas as pd
import requests

def get_datafields(s, searchScope, dataset_id: str = '', search: str = ''):
    instrument_type = searchScope['instrumentType']
    region = searchScope['region']
    delay = searchScope['delay']
    universe = searchScope['universe']
    
    if len(search) == 0:
        url_template = f"https://api.worldquantbrain.com/data-fields?instrumentType={instrument_type}&region={region}&delay={str(delay)}&universe={universe}&dataset.id={dataset_id}&limit=50&offset={{x}}"
        count = 100
    else:
        url_template = f"https://api.worldquantbrain.com/data-fields?instrumentType={instrument_type}&region={region}&delay={str(delay)}&universe={universe}&limit=50&search={search}&offset={{x}}"
        count = 100

    datafields_list = []
    for x in range(0, count, 50):
        datafields = s.get(url_template.format(x=x))
        if datafields.status_code == 200:
            datafields_list.append(datafields.json()['results'])
        else:
            print(f"Error fetching data at offset {x}: {datafields.status_code}")
    
    datafields_list_flat = [item for sublist in datafields_list for item in sublist]
    
    datafields_df = pd.DataFrame(datafields_list_flat)
    
    return datafields_df

In [4]:
import pandas as pd
import requests

def get_datafields(s, searchScope, dataset_id: str = '', search: str = ''):
    instrument_type = searchScope['instrumentType']
    region = searchScope['region']
    delay = searchScope['delay']
    universe = searchScope['universe']
    
    if len(search) == 0:
        url_template = "https://api.worldquantbrain.com/data-fields?" \
            f"&instrumentType={instrument_type}" \
            f"&region={region}&delay={str(delay)}&universe={universe}&dataset.id={dataset_id}&limit=50" \
            "&offset={x}"
        count = s.get(url_template.format(x=0)).json()['count']
    else:
        url_template = "https://api.worldquantbrain.com/data-fields?" \
            f"&instrumentType={instrument_type}" \
            f"&region={region}&delay={str(delay)}&universe={universe}&limit=50" \
            f"&search={search}" \
            "&offset={x}"
        count = 100

    datafields_list = []
    for x in range(0, count, 50):
        datafields = s.get(url_template.format(x=x))
        datafields_list.append(datafields.json()['results'])
    
    datafields_list_flat = [item for sublist in datafields_list for item in sublist]
    
    datafields_df = pd.DataFrame(datafields_list_flat)
    return datafields_df

In [5]:
searchScope = {'region': 'USA', 'delay': 1, 'universe': 'TOP3000','instrumentType': 'EQUITY'}
fundamental6 = get_datafields(s = sess, searchScope = searchScope, dataset_id = 'fundamental6')

In [6]:
fundamental6 = fundamental6[fundamental6['type'] == 'MATRIX']
fundamental6.head()

,id,description,dataset,category,subcategory,region,delay,universe,type,coverage,userCount,alphaCount,themes
0,assets,Assets - Total,"{'id': 'fundamental6', 'name': 'Company Fundam...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",USA,1,TOP3000,MATRIX,0.9524,27516,97405,[]
1,assets_curr,Current Assets - Total,"{'id': 'fundamental6', 'name': 'Company Fundam...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",USA,1,TOP3000,MATRIX,0.7655,2568,12954,[]
2,bookvalue_ps,Book Value Per Share,"{'id': 'fundamental6', 'name': 'Company Fundam...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",USA,1,TOP3000,MATRIX,0.9754,2059,8822,[]
3,capex,Capital Expenditures,"{'id': 'fundamental6', 'name': 'Company Fundam...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",USA,1,TOP3000,MATRIX,0.9646,8449,21865,[]
4,cash,Cash,"{'id': 'fundamental6', 'name': 'Company Fundam...","{'id': 'fundamental', 'name': 'Fundamental'}","{'id': 'fundamental-fundamental-data', 'name':...",USA,1,TOP3000,MATRIX,0.7529,1926,10535,[]


In [7]:
datafields_list_fundamental6 = fundamental6['id'].values

In [8]:
datafields_list_fundamental6

array(['assets', 'assets_curr', 'bookvalue_ps', 'capex', 'cash',
       'cash_st', 'cashflow', 'cashflow_dividends', 'cashflow_fin',
       'cashflow_invst', 'cashflow_op', 'cogs', 'current_ratio', 'debt',
       'debt_lt', 'debt_st', 'depre_amort', 'ebit', 'ebitda', 'employee',
       'enterprise_value', 'eps', 'equity', 'fnd6_acdo', 'fnd6_acodo',
       'fnd6_acox', 'fnd6_acqgdwl', 'fnd6_acqintan',
       'fnd6_adesinda_curcd', 'fnd6_aldo', 'fnd6_am', 'fnd6_aodo',
       'fnd6_aox', 'fnd6_aqc', 'fnd6_aqi', 'fnd6_aqs', 'fnd6_beta',
       'fnd6_capxv', 'fnd6_ceql', 'fnd6_ch', 'fnd6_ci', 'fnd6_cibegni',
       'fnd6_cicurr', 'fnd6_cidergl', 'fnd6_cik', 'fnd6_cimii',
       'fnd6_ciother', 'fnd6_cipen', 'fnd6_cisecgl', 'fnd6_citotal',
       'fnd6_city', 'fnd6_cld2', 'fnd6_cld3', 'fnd6_cld4', 'fnd6_cld5',
       'fnd6_cptmfmq_actq', 'fnd6_cptmfmq_atq', 'fnd6_cptmfmq_ceqq',
       'fnd6_cptmfmq_dlttq', 'fnd6_cptmfmq_dpq', 'fnd6_cptmfmq_lctq',
       'fnd6_cptmfmq_oibdpq', 'fnd6_cptmfmq_o

In [9]:
alpha_list = []

for i, datafield in enumerate(datafields_list_fundamental6):
    print(f"正在将如下Alpha表达式与setting封装")
    alpha_expression = f"group_rank({datafield}/cap, subindustry)"
    print(i, alpha_expression)
    simulation_data = {
        'type': 'REGULAR',
        'settings': {
            'instrumentType': 'EQUITY',
            'region': 'USA',
            'universe': 'TOP3000',
            'delay': 1,
            'decay': 0,
            'neutralization': 'SUBINDUSTRY',
            'truncation': 0.08,
            'pasteurization': 'ON',
            'unitHandling': 'VERIFY',
            'nanHandling': 'ON',
            'language': 'FASTEXPR',
            'visualization': False,
        },
        'regular': alpha_expression
    }
    
    alpha_list.append(simulation_data)

print(f"there are {len(alpha_list)} Alphas to simulate")

正在将如下Alpha表达式与setting封装
0 group_rank(assets/cap, subindustry)
正在将如下Alpha表达式与setting封装
1 group_rank(assets_curr/cap, subindustry)
正在将如下Alpha表达式与setting封装
2 group_rank(bookvalue_ps/cap, subindustry)
正在将如下Alpha表达式与setting封装
3 group_rank(capex/cap, subindustry)
正在将如下Alpha表达式与setting封装
4 group_rank(cash/cap, subindustry)
正在将如下Alpha表达式与setting封装
5 group_rank(cash_st/cap, subindustry)
正在将如下Alpha表达式与setting封装
6 group_rank(cashflow/cap, subindustry)
正在将如下Alpha表达式与setting封装
7 group_rank(cashflow_dividends/cap, subindustry)
正在将如下Alpha表达式与setting封装
8 group_rank(cashflow_fin/cap, subindustry)
正在将如下Alpha表达式与setting封装
9 group_rank(cashflow_invst/cap, subindustry)
正在将如下Alpha表达式与setting封装
10 group_rank(cashflow_op/cap, subindustry)
正在将如下Alpha表达式与setting封装
11 group_rank(cogs/cap, subindustry)
正在将如下Alpha表达式与setting封装
12 group_rank(current_ratio/cap, subindustry)
正在将如下Alpha表达式与setting封装
13 group_rank(debt/cap, subindustry)
正在将如下Alpha表达式与setting封装
14 group_rank(debt_lt/cap, subindustry)
正在将如下Alpha表达式与setti

In [ ]:
from time import sleep

start_index = 87
for i, alpha in enumerate(alpha_list):
    print(i, alpha)
    if i < start_index:
        print("skip", i, alpha)
        continue
    sim_resp = sess.post(
        'https://api.worldquantbrain.com/simulations',
        json=alpha,
    )
    
    try:
        sim_progress_url = sim_resp.headers['location']
        while True:
            sim_progress_resp = sess.get(sim_progress_url)
            retry_after_sec = float(sim_progress_resp.headers.get("Retry-After", 0))
            if retry_after_sec == 0:  # simulation done!
                break
            sleep(retry_after_sec)
        
        alpha_id = sim_progress_resp.json()["alpha"]  # the final simulation result
        print(alpha_id)
        
    except:
        print("no location, sleep for 10 seconds and try next alpha")
        sleep(10)
        sess = get_auth()

0 {'type': 'REGULAR', 'settings': {'instrumentType': 'EQUITY', 'region': 'USA', 'universe': 'TOP3000', 'delay': 1, 'decay': 0, 'neutralization': 'SUBINDUSTRY', 'truncation': 0.08, 'pasteurization': 'ON', 'unitHandling': 'VERIFY', 'nanHandling': 'ON', 'language': 'FASTEXPR', 'visualization': False}, 'regular': 'group_rank(assets/cap, subindustry)'}
skip 0 {'type': 'REGULAR', 'settings': {'instrumentType': 'EQUITY', 'region': 'USA', 'universe': 'TOP3000', 'delay': 1, 'decay': 0, 'neutralization': 'SUBINDUSTRY', 'truncation': 0.08, 'pasteurization': 'ON', 'unitHandling': 'VERIFY', 'nanHandling': 'ON', 'language': 'FASTEXPR', 'visualization': False}, 'regular': 'group_rank(assets/cap, subindustry)'}
1 {'type': 'REGULAR', 'settings': {'instrumentType': 'EQUITY', 'region': 'USA', 'universe': 'TOP3000', 'delay': 1, 'decay': 0, 'neutralization': 'SUBINDUSTRY', 'truncation': 0.08, 'pasteurization': 'ON', 'unitHandling': 'VERIFY', 'nanHandling': 'ON', 'language': 'FASTEXPR', 'visualization': Fal